# Ngram Spacing and Smoothing

### José Pablo Kiesling Lange

In [1]:
import random
import re
from collections import Counter
import pandas as pd
import math

import nltk
from nltk.corpus import cess_esp
from nltk.util import ngrams
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline

In [2]:
random.seed(1234)

In [3]:
nltk.download('cess_esp')
nltk.download('punkt')

[nltk_data] Downloading package cess_esp to
[nltk_data]     C:\Users\TheKi\AppData\Roaming\nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TheKi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
corpus_nltk = cess_esp.words()

## Estandarización

Para ver la efectivdad de la estandarización, se mostrará las 10 palabras más frecuentes del corpus antes y después de la estandarización. El objetivo es ver si hay modificación en la cantidad de dichas palabras o si una nueva palabra aparece con frecuencia.

In [5]:
corpus = []

In [6]:
def get_most_common_words(corpus, n=10):
    words = [word for line in corpus for word in line.split()]
    most_common = Counter(words).most_common(n)
    return most_common

In [7]:
most_common_words = get_most_common_words(corpus_nltk, n=10)
most_common_words

[(',', 11420),
 ('de', 10234),
 ('la', 6412),
 ('.', 5866),
 ('que', 5552),
 ('el', 5199),
 ('en', 4340),
 ('y', 4235),
 ('*0*', 3883),
 ('"', 3038)]

Además se mostrará los primeros 25 tokens del corpus antes y después de la estandarización.

In [8]:
corpus_nltk[:25]

['El',
 'grupo',
 'estatal',
 'Electricité_de_France',
 '-Fpa-',
 'EDF',
 '-Fpt-',
 'anunció',
 'hoy',
 ',',
 'jueves',
 ',',
 'la',
 'compra',
 'del',
 '51_por_ciento',
 'de',
 'la',
 'empresa',
 'mexicana',
 'Electricidad_Águila_de_Altamira',
 '-Fpa-',
 'EAA',
 '-Fpt-',
 ',']

Como se puede apreciar, hay palabras que tienen `_` en los tokens y separan palabras. Por lo que se harán las funciones específicas para limpiar los tokens y separar las palabras.

In [9]:
def replace_whitespaces(word):
    return "".join(re.sub('_', ' ', word))

In [10]:
def separate_words(word):
    return word.split()

In [11]:
corpus_without_underscore = [replace_whitespaces(word) for word in corpus_nltk]

In [12]:
for word in corpus_without_underscore:
    if len(word.split()) > 1:
        corpus.extend(separate_words(word))
    else:
        corpus.append(word)

Además, se puede ver que hay tokens que empiezan con caracteres no alfanuméricos o que contienen caracteres especiales. Específicamente, los siguientes

In [13]:
list(set(word for word in corpus if not word.isalnum()))[:10]

['Reynard/Toyota',
 '07.30.42',
 '164,4',
 '110,37',
 '5-1',
 'CFE-CGC',
 'G.P.',
 '2.500',
 'Moet-Hennessy',
 '1,96']

Como se puede ver, hay palabras que tienen `'`o `"` en los tokens por lo que *solo* se eliminarán esos caracteres y no el resto del token. En los otros casos, no representan alguna palabra, por lo que se eliminarán completamente.

In [14]:
def clean_word(word):
    if re.search(r'[^a-zA-Z0-9\'"áéíóúÁÉÍÓÚ]', word):
        return ""
    
    return re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚ]', '', word)

In [15]:
corpus = [clean_word(word) for word in corpus]

In [16]:
corpus = [word for word in corpus if word != '']

Finalmente, se pondrán las palabras en minúsculas solo si no es sigla o acrónimo. Para esto, se hará una función que verifique si la palabra está en mayúsculas y si no es así, la pondrá en minúsculas.

In [17]:
def case_folding(corpus):
    return [word.lower() if not word.isupper() else word for word in corpus]

In [18]:
corpus = case_folding(corpus)

### Resultado

In [19]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('de', 11828),
 ('la', 7150),
 ('el', 6079),
 ('que', 5943),
 ('en', 4991),
 ('y', 4318),
 ('a', 3493),
 ('los', 3229),
 ('del', 2514),
 ('las', 1956)]

In [20]:
corpus[:25]

['el',
 'grupo',
 'estatal',
 'electricité',
 'de',
 'france',
 'EDF',
 'anunció',
 'hoy',
 'jueves',
 'la',
 'compra',
 'del',
 '51',
 'por',
 'ciento',
 'de',
 'la',
 'empresa',
 'mexicana',
 'electricidad',
 'águila',
 'de',
 'altamira',
 'EAA']

Como se puede apreciar, ya solo hay palabras alfanuméricas y con acentos. Y todas están separadas como se debe

## Construcción de n-gramas

In [21]:
tokens = nltk.word_tokenize(' '.join(corpus), language='spanish')

In [22]:
random.shuffle(tokens)
split = int(0.8 * len(tokens))

In [23]:
train = tokens[:split]
test  = tokens[split:]

In [24]:
def generate_ngrams(tokens, n):
    ngram = Counter( ngrams(tokens, n))
    return ngram

In [25]:
def mle_ngram_probs(tokens, k):   
    ngram = generate_ngrams(tokens, k)
     
    if k == 1:
        total = sum(ngram.values())
        return { (w,): c/total for (w,), c in ngram.items() }
    
    history_counts = Counter(ngrams(tokens, k-1))
    probs = {}
    for kgram, c in ngram.items():
        history = kgram[:-1]
        probs[kgram] = c / history_counts[history]
        
    return probs

In [26]:
p1 = mle_ngram_probs(train, 1)
p2 = mle_ngram_probs(train, 2)
p3 = mle_ngram_probs(train, 3)

## Análisis de espacidad

In [27]:
def sparsity(train, test):
    seen = { n: set(ngrams(train, n)) for n in range(1, 4) }
    results = {}
    
    for n in range(1, 4):
        t_ngrams = list(ngrams(test, n))
        unseen = sum(1 for ng in t_ngrams if ng not in seen[n])
        results[n] = 100 * unseen / len(t_ngrams) if t_ngrams else 0

    return results

In [28]:
pct_unseen = sparsity(train, test)
pct_unseen_df = pd.Series(pct_unseen).rename_axis('n').reset_index(name='% no vistos')
pct_unseen_df

,n,% no vistos
0,1,7.718238
1,2,65.638895
2,3,94.373729


Como se puede apreciar, mientras más cantidad de tokens se toma en cuenta para el contexto (n), el porcentaje de ngramas no vistos aumenta. Esto es porque al aumentar el contexto, se reduce la cantidad de ngramas que se pueden generar, ya que se requiere más información para formar un ngrama.

## Suavizado

### Laplace

In [29]:
def laplace(tokens, k):
    vocab_size = len(set(tokens))
    
    ngram = generate_ngrams(tokens, k)
    
    if k == 1:
        total = sum(ngram.values())
        return { (w,): (c + 1) / (total + vocab_size) for (w,), c in ngram.items() }
    
    history_counts = Counter(ngrams(tokens, k-1))
    probs = {}
    for kgram, c in ngram.items():
        history = kgram[:-1]
        probs[kgram] = (c + 1) / (history_counts[history] + vocab_size)
        
    return probs

In [30]:
p1_laplace = laplace(train, 1)
p2_laplace = laplace(train, 2)
p3_laplace = laplace(train, 3)

### Interpolación Lineal

In [31]:
lambdas = [1/3, 1/3, 1/3]

In [32]:
def linear_interpolation(tokens, k):
    all_probs = {}
    for n in range(1, k+1):
        all_probs[n] = mle_ngram_probs(tokens, n)

    ngram_k = generate_ngrams(tokens, k)
    interpolated_probs = {}
    
    for kgram in ngram_k.keys():
        prob = 0.0
        for n in range(1, k+1):
            subgram = kgram[-n:] if n <= len(kgram) else kgram
            if subgram in all_probs[n]:
                prob += lambdas[n-1] * all_probs[n][subgram]
        interpolated_probs[kgram] = prob
    
    return interpolated_probs

In [33]:
p2_interpolated = linear_interpolation(train, 2)
p3_interpolated = linear_interpolation(train, 3)

### Kneser-Ney

In [34]:
train_data, padded_vocab = padded_everygram_pipeline(3, train)
kn_model = KneserNeyInterpolated(3)
kn_model.fit(train_data, padded_vocab)

## Perplejidad

In [35]:
def calculate_perplexity(test_tokens, prob_dict, n, smoothing_type, model=None, sample_size=500):
    test_ngrams = list(ngrams(test_tokens, n))
    
    log_prob_sum = 0
    
    if smoothing_type == 'kneser_ney':
        for ngram in test_ngrams:
            try:
                context = ngram[:-1] if n > 1 else []
                word = ngram[-1]
                prob = model.score(word, context)
                log_prob_sum += math.log(prob)
            except:
                log_prob_sum += math.log(1e-6)
    else:
        vocab_size = len(set(train))
        default_prob = 1e-6
        
        if smoothing_type == 'laplace':
            default_prob = 1 / (vocab_size * 2)
        
        for ngram in test_ngrams:
            prob = prob_dict.get(ngram, default_prob)
            log_prob_sum += math.log(max(prob, 1e-6))
    
    if len(test_ngrams) > 0:
        avg_log_prob = log_prob_sum / len(test_ngrams)
        return math.exp(-avg_log_prob)
    else:
        return float('inf')

In [36]:
results = []

In [37]:
for n, prob_dict in [(1, p1_laplace), (2, p2_laplace), (3, p3_laplace)]:
    perplexity = calculate_perplexity(test, prob_dict, n, 'laplace')
    results.append({
        'Modelo': f'{n}-grama',
        'Técnica': 'Laplace (Add-1)',
        'Perplejidad': perplexity
    })

In [38]:
for n, prob_dict in [(2, p2_interpolated), (3, p3_interpolated)]:
    perplexity = calculate_perplexity(test, prob_dict, n, 'interpolated')
    results.append({
        'Modelo': f'{n}-grama',
        'Técnica': 'Interpolación lineal',
        'Perplejidad': perplexity
    })


In [39]:
kn_perplexity = calculate_perplexity(test, None, 3, 'kneser_ney', kn_model)
kn_perplexity = kn_perplexity if kn_perplexity != float('inf') else 'Inf'
results.append({
    'Modelo': '3-grama',
    'Técnica': 'Kneser-Ney',
    'Perplejidad': kn_perplexity if kn_perplexity != float('inf') else 'Inf'
})


In [40]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Perplejidad')
results_df

,Modelo,Técnica,Perplejidad
0,1-grama,Laplace (Add-1),1192.103956
1,2-grama,Laplace (Add-1),12791.897088
2,3-grama,Laplace (Add-1),34595.715186
3,2-grama,Interpolación lineal,49230.063398
4,3-grama,Interpolación lineal,558977.323530
5,3-grama,Kneser-Ney,607222.414957
